In [2]:
import warnings;
warnings.filterwarnings('ignore')

In [3]:
import pickle
with open('../data/clean-data-nostem.pkl','rb') as fp:
    X,Y = pickle.load(fp)
fp.close()

In [4]:
l = int(0.8*len(X))

### Class for Tfidf weighted W2V

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np

In [6]:
# Code taken from [http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/]
# This uses tfidf weighted w2v
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.values())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

### Tokenizing

In [7]:
X = [i.split() for i in X]

### W2V without stemming
Since it gave a better result than stemmed data

In [8]:
import gensim
# let X be a list of tokenized texts (i.e. list of lists of tokens)
model = gensim.models.Word2Vec(X, size=100)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [10]:
vectorizer = TfidfEmbeddingVectorizer(w2v)

In [12]:
vectorizer.fit(X[:l],Y[:l])

In [15]:
tfidf_w2v_tr = vectorizer.transform(X[:l])

In [18]:
tfidf_w2v_ts = vectorizer.transform(X[l:])